In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

from simulator import Simulation

import jax.numpy as jnp
from jax import grad, value_and_grad
import numpy as np
from scipy.optimize import minimize
import numpy as np

warnings.simplefilter(action='ignore', category=UserWarning)

In [8]:
def simulate_base(quorum, min_amount, max_funding=30_000_000, num_voters=150, num_projects=643, num_mc=10):
    simulation = Simulation()

    simulation.initialize_round(
        max_funding=max_funding
    )
    simulation.randomize_voters(
        num_voters=num_voters, 
        willingness_to_spend=1.0, 
        laziness_factor=0.5, 
        expertise_factor=0.5
    )
    simulation.randomize_projects(
        num_projects=num_projects
    )

    results = simulation.simulate_voting_and_scoring(
        n=num_mc,
        scoring_method='median',
        quorum=quorum,
        min_amount=min_amount,
        normalize=True
    )
    # extract the results of interest to optimize
    # create an aligned array of rating --> token_amount
    data = results['data']
    rating_arr = np.array([d['rating'] for d in data])
    token_amount_arr = np.array([d['token_amount'] for d in data])

    # normalize these so we can optimize
    rating_arr = (rating_arr - rating_arr.min()) / (rating_arr.max() - rating_arr.min())
    token_amount_arr = (token_amount_arr - token_amount_arr.min()) / (token_amount_arr.max() - token_amount_arr.min())

    return rating_arr, token_amount_arr

def loss_fn(quorum, min_amount=1500, max_funding=30_000_000, num_voters=150, num_projects=643, num_mc=10):
    rating_arr, token_amount_arr = simulate_base(quorum, min_amount, max_funding, num_voters, num_projects, num_mc)
    mse = np.sum((rating_arr - token_amount_arr) ** 2)
    return mse

def loss_full(quorum, min_amount, max_funding=30_000_000, num_voters=150, num_projects=643, num_mc=10):
    rating_arr, token_amount_arr = simulate_base(quorum, min_amount, max_funding, num_voters, num_projects, num_mc)
    mse = np.sum((rating_arr - token_amount_arr) ** 2)
    return mse, rating_arr, token_amount_arr

In [9]:
from scipy.optimize import minimize_scalar
minimize_scalar(loss_fn, bounds=(1,30), options={'maxiter': 1000, 'disp':True})


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


 message: Solution found.
 success: True
  status: 0
     fun: 57.594779679043796
       x: 16.394452868410852
     nit: 32
    nfev: 32

In [ ]:
initial_params = np.array([10.0, 1000.0])
result = minimize(loss_fn, initial_params, 
                  bounds=[(1, 30), (200, 2000)], 
                  options={'disp': True, 'maxiter': 10})
optimized_params = result.x

In [ ]:
result

In [ ]:
configs = [
    (1, 1),
    (5, 200),
    (17, 1500),
    (100, 2500),
]
config2results = {}
for c in configs:
    mse, rating_arr, token_amount_arr = loss_full(c)
    config2results[c] = (rating_arr, token_amount_arr, mse)

In [ ]:
plt.figure()
for c in configs:
    rating_arr, token_amount_arr, loss_val = config2results[c]
    plt.scatter(rating_arr, token_amount_arr, label=f"{loss_val}")
plt.legend()
plt.xlabel('Impact')
plt.ylabel('Token Allocation')

In [ ]:
# initialize to known values
params = jnp.asarray([17.0, 1500.0])

# Compute gradient of the loss function
grad_loss_fn = value_and_grad(loss)

# Optimize the parameters using gradient descent
learning_rate = 0.1
num_iterations = 10

for i in range(num_iterations):
    mse, gradient = grad_loss_fn(params)
    params -= learning_rate * gradient
    print(mse, gradient, params)

print("Optimized parameters (m, c):", params)

In [ ]:
quorum = 17
min_amount = 1500
rating_arr, token_amounts_arr = simulate_base((quorum, min_amount), num_mc=10)

In [ ]:
plt.figure()
plt.scatter(rating_arr, token_amounts_arr)